In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Đọc file
df = pd.read_csv("Test_Qwen2.5_3B_Instruct.csv")

# =======================
# ==  Stage 1: Nhận diện sai lệch ==
# =======================
y_true_stage1 = df["label"]
y_pred_stage1 = df["pred_label"]

acc_stage1 = accuracy_score(y_true_stage1, y_pred_stage1)
f1_stage1 = f1_score(y_true_stage1, y_pred_stage1, average='binary')

report_stage1 = classification_report(
    y_true_stage1,
    y_pred_stage1,
    digits=2
)

print("== Stage 1: Nhận diện sai lệch ==")
print(f"Accuracy: {acc_stage1}")
print(f"F1 Score: {f1_stage1}")
print(report_stage1)

== Stage 1: Nhận diện sai lệch ==
Accuracy: 0.9848275862068966
F1 Score: 0.991747936984246
              precision    recall  f1-score   support

           0       0.88      0.93      0.91        57
           1       0.99      0.99      0.99       668

    accuracy                           0.98       725
   macro avg       0.94      0.96      0.95       725
weighted avg       0.99      0.98      0.99       725



In [2]:
df_stage2 = df[df["label"] == 1].copy()

# Loại bỏ các dòng có NaN ở bias_type hoặc pred_bias
df_stage2 = df_stage2.dropna(subset=["bias_type", "pred_bias"])

y_true_stage2 = df_stage2["bias_type"].astype(str)
y_pred_stage2 = df_stage2["pred_bias"].astype(str)

acc_stage2 = accuracy_score(y_true_stage2, y_pred_stage2)
f1_stage2_macro = f1_score(y_true_stage2, y_pred_stage2, average='macro')

report_stage2 = classification_report(
    y_true_stage2,
    y_pred_stage2,
    digits=2
)

print("\n== Stage 2: Phân loại loại sai lệch ==")
print(f"Accuracy: {acc_stage2}")
print(f"F1 Score (macro): {f1_stage2_macro}")
print(report_stage2)


== Stage 2: Phân loại loại sai lệch ==
Accuracy: 0.9016641452344932
F1 Score (macro): 0.9004766842484656
                                precision    recall  f1-score   support

       All-or-Nothing Thinking       0.84      0.86      0.85        78
    Disqualifying the Positive       0.85      0.95      0.90        80
           Emotional Reasoning       0.96      0.97      0.97        71
        Jumping to Conclusions       0.90      0.91      0.91        79
      Labeling and Mislabeling       0.93      0.90      0.91        89
Magnification and Minimization       0.93      0.76      0.84        55
                 Mental Filter       0.93      0.94      0.93        53
            Overgeneralization       0.85      0.80      0.83        51
               Personalization       0.89      0.94      0.92        54
             Should Statements       0.96      0.94      0.95        51

                      accuracy                           0.90       661
                     macro a

In [6]:
samples = df.sample(8)

def format_example(row, idx):
    text = f"""
### ✔ Ví dụ {idx}

**🔹 Đoạn hội thoại:**
{row['dialogue']}

**🔹 Nhãn thật:**
- Có sai lệch không: **{row['label']}**
- Loại sai lệch: **{row['bias_type']}**
- Giải thích: {row['explanation']}

**🔹 Mô hình dự đoán:**
- Có sai lệch không: **{row['pred_label']}**
- Loại sai lệch: **{row['pred_bias']}**
- Giải thích mô hình: {row['pred_explanation']}

---
"""
    return text

output = ""
for i, (_, row) in enumerate(samples.iterrows(), start=1):
    output += format_example(row, i)

print(output)


### ✔ Ví dụ 1

**🔹 Đoạn hội thoại:**
A: Ôi không, con hamster của tớ lại mất rồi. Sao lúc nào nuôi thú cưng cũng vậy nhỉ? Chắc cả đời này thú cưng của tớ sẽ cứ biến mất bí ẩn thôi.
B: Thôi mà, đừng nghĩ quá nghiêm trọng. Nó có khi chỉ trốn đâu đó thôi mà. Với lại, đâu phải lần nào cũng mất đâu.
A: Không, tớ thấy rõ ràng là thế. Nuôi con nào là mất con đó. Chắc đây là số phận của tớ rồi.
B: Cậu đang khái quát hóa vài lần mất thú cưng thành quy luật cho cả đời mình đó.

**🔹 Nhãn thật:**
- Có sai lệch không: **1**
- Loại sai lệch: **Overgeneralization**
- Giải thích: Người nói khái quát hóa từ một vài lần mất thú cưng để kết luận rằng 'cả đời này thú cưng của tôi sẽ biến mất một cách bí ẩn'. Đây là biểu hiện của Overgeneralization.

**🔹 Mô hình dự đoán:**
- Có sai lệch không: **1**
- Loại sai lệch: **Overgeneralization**
- Giải thích mô hình: Người nói khái quát hóa từ vài lần thú cưng mất mát thành kết luận rằng 'thú cưng sẽ cứ biến mất bí ẩn' trong suốt cuộc đời, thể hiện Overgeneraliz